In [1]:
import pandas as pd
import numpy as np 
import re
from textblob import TextBlob
from nltk.corpus import stopwords 
from sklearn import model_selection, preprocessing, naive_bayes, metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import string
from sklearn.externals import joblib
from sklearn import svm
import os
from sklearn.model_selection import KFold 
current_path = os.getcwd()
current_path

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


'/Users/rhea-8992/git_repo_changes made/Text'

In [2]:
class Sentiment_analysis:  
    
    # A pre-processing step to make the texts cleaner and easier to process and a vectorization step to transform these texts into numerical vectors.
    def data_prep(self):
        df = open("amazon_reviews.csv").read() 
        data = df.split("\n")
        labels, texts = [], []
        for i, line in enumerate(data):
            content = line.split()
            labels.append(content[0])
            texts.append(" ".join(content[1:]))
        d = {'Label':labels,'Reviews':texts}
        df1 = pd.DataFrame(d)
        return df1
    
    def data_preprocess(self): 
        df2 = self.data_prep()
        # Removal of stopwords
        stop = stopwords.words('english')
        df2["Reviews"] = df2["Reviews"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
        # Converting the reviews into lowercase
        df2["Reviews"] = df2["Reviews"].apply(lambda x: " ".join(x.lower() for x in x.split()))
        # Removing punctuation marks 
        df1["Reviews"] = df1["Reviews"].str.replace('[^\w\s]',' ')
        # Rare words removal 
        freq = pd.Series(' '.join(df2["Reviews"]).split()).value_counts()[-10:]
        freq = list(freq.index)
        df2["Reviews"] = df2["Reviews"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
        # Tokenization of text data 
        Token_list = []
        for i in range(0, len(df2)):
            Token_list.append(TextBlob(df2["Reviews"][i]).words)
        return Token_list

    def split_data(self):
        df2 = self.data_prep()
        #splitting the dataset into training and validation datasets
        train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df2["Reviews"], df2["Label"], random_state = 42, test_size = 0.2)
        # label encode the target variable
        encoder = preprocessing.LabelEncoder()
        train_y = encoder.fit_transform(train_y)
        valid_y = encoder.fit_transform(valid_y)
        count_vect = CountVectorizer(analyzer = 'word', token_pattern = r'\w{1,}')
        count_vect.fit(df2["Reviews"])
        #transform the training and validation data using count vectorizer object 
        xtrain_count =  count_vect.transform(train_x)
        xvalid_count =  count_vect.transform(valid_x)
        return xtrain_count, xvalid_count, train_y, valid_y, valid_x
   
    def train_model(self, classifier, feature_vector_train, label, feature_vector_valid ,valid_y):
        # fit the training dataset on the classifier 
        t = feature_vector_train.toarray()
        classifier.fit(t, label )
        # predict the labels on validation dataset 
        predictions = classifier.predict(feature_vector_valid)
        return metrics.accuracy_score(predictions, valid_y), metrics.f1_score(predictions, valid_y), classifier
    
    #MultinomialNB()
    def prediction(self):
        xtrain_count, xvalid_count, train_y, valid_y, valid_x = self.split_data()
        accuracy, f1_score, model = self.train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
        return accuracy, f1_score, model
    
    #BernoulliNB()
    def prediction1(self):
        xtrain_count, xvalid_count, train_y, valid_y, valid_x = self.split_data()
        accuracy, f1_score, model = self.train_model(naive_bayes.BernoulliNB(), xtrain_count, train_y, xvalid_count, valid_y)
        return accuracy, f1_score, model
    
    #SVM
    def prediction2(self):
        xtrain_count, xvalid_count, train_y, valid_y, valid_x = self.split_data()
        accuracy, f1_score, model = self.train_model(svm.LinearSVC(), xtrain_count, train_y, xvalid_count, valid_y)
        return accuracy, f1_score, model
    
    

In [3]:
sent_obj = Sentiment_analysis()

In [4]:
#NAIVE BAYES - MULTINOMIAL 
accuracy, f1_score, model = sent_obj.prediction()

In [5]:
#NAIVE BAYES - MULTINOMIAL
accuracy, f1_score

(0.829, 0.814935064935065)

In [6]:
#NAIVE BAYES - BERNOULLI
accuracy1, f1_score1, model1 = sent_obj.prediction1()

In [7]:
#NAIVE BAYES - BERNOULLI
accuracy1, f1_score1

(0.838, 0.8287526427061311)

In [8]:
#SVM
accuracy2, f1_score2, model2 = sent_obj.prediction2()

In [9]:
#SVM 
accuracy2, f1_score2

(0.8385, 0.8323819408406851)